# Saving the face mesh

In this script, I generate the output of the Volumetric Regression Network (VRN) found here:
https://github.com/AaronJackson/vrn

In [1]:
from keras.models import load_model
import cv2
import visvis as vv
import numpy as np
from skimage import measure

import custom_layers

Using TensorFlow backend.


#### Load the model I ported from torch7 to keras.

In [2]:
custom_objects = {
    'Conv': custom_layers.Conv,
    'BatchNorm': custom_layers.BatchNorm,
    'UpSamplingBilinear': custom_layers.UpSamplingBilinear
}
model = load_model('vrn-unguided-keras.h5', custom_objects=custom_objects)

In [3]:
def load_img(file_name):
    img = cv2.imread(file_name)
    if "tron" in file_name:
        img = img[115:640, 763:1287, :]
    img = cv2.resize(img, (192, 192))
    img = cv2.flip(img, 1)
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    img = np.swapaxes(img, 2, 0)
    img = np.swapaxes(img, 2, 1)
    img = np.array([img])
    return img

#### Render the mesh and save the frame.

In [4]:
def screenshot(pred, img, output_file_name, blue=True):
    vol = pred[0] * 255
    vol = np.swapaxes(vol, 0, 2)

    verts, faces, normals, values = measure.marching_cubes_lewiner(vol, 1)
        
    vv.clf()
        
    im = img[0]
    im = np.swapaxes(im, 0, 1)
    im = np.swapaxes(im, 1, 2)
    t = vv.imshow(im)
    t.interpolate = True
    
    colormap = None
    texture = None
    if blue:
        colormap = [[.5, .5, 1] for z, y, x in verts]
    else:
        w = float(im.shape[1])
        h = float(im.shape[0])
        texture = [[x / w, y / h] for x, y, z in verts]

    vvmesh = vv.mesh(verts, 
                     faces=faces, 
                     normals=normals, 
                     values=colormap if blue else texture, texture=im)

    a = vv.gca()
    a.camera.fov = 0 # orthographic
    a.camera.azimuth = 180
    a.camera.elevation = 90
    a.camera.roll = 0
    a.daspect = (1, 1, .5)

    if output_file_name is not None:
        vv.screenshot(output_file_name, vv.gcf(), sf=2, bg='w')
    else:
        vv.use().Run()

#### Iterate through the video, getting a prediction for each frame.

In [ ]:
meshes = []
for i in range(1, 101):
    img = load_img("/Users/paul/Downloads/tron_frames/frame{:04d}.jpg".format(i))
    pred = model.predict(img)
    screenshot(pred, img, "tron_output/frame{:04d}.jpg".format(i))

I'll save the mesh that I will use later in the face swap script.

In [75]:
img = load_img("joey.jpg")
pred = model.predict(img)
vol = pred[0] * 255
vol = np.swapaxes(vol, 0, 2)
verts, faces, normals, values = measure.marching_cubes_lewiner(vol, 1)
mesh = np.array((verts, faces, normals, values))
np.save('joey_mesh.npy', mesh)